In [ ]:
import os
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Input
# from tensorflow.keras.callbacks import EarlyStopping
# import keras_tuner as kt

In [ ]:
# GPU memory configuration
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
# Load and preprocess dataset
model_dir = 'Model_architecture/FCNN_v1/'
os.makedirs(model_dir, exist_ok=True)

variables_to_predict = 4 # CP , CF_x, CF_y, CF_z
shuffle_data = False

# Flags for training and loading
Transfer_Learning_flag = True
Execute_Train_flag = False

dataset = np.load('Dataset\\dataset.npy')

print( '\n-> Shape of the loaded matrix: \n')
print(dataset.shape) # x, y , z , CP , CF_x, CF_y, CF_z, M , AoA

X = dataset[:,:,[0,1,2,7,8]] # Input
Y = dataset[:,:,3:7]         # Output

print(f"\nInput matrix shape: {X.shape} \nOutput matrix shape: {Y.shape}")

In [ ]:
## Feature normalisation:
scaler = MinMaxScaler(feature_range=(-1, 1))
samples, points, variables = X.shape
X = np.reshape(X, newshape=(-1, variables))
X = scaler.fit_transform(X)
X = np.reshape(X, newshape=(samples, points, variables))

scalery = MinMaxScaler(feature_range=(-1, 1))
samples, points, variables = Y.shape
Y = np.reshape(Y, newshape=(-1, variables))
Y = scalery.fit_transform(Y)
Y = np.reshape(Y, newshape=(samples, points, variables))

In [ ]:
# Split data
train_ratio, test_ratio, val_ratio = 0.58, 0.20, 0.20
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=1 - train_ratio, shuffle=shuffle_data)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=test_ratio / (test_ratio + val_ratio), shuffle=shuffle_data)

In [ ]:
# Model definition using Keras Tuner
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(X.shape[1], X.shape[2])))
    prelu = tf.keras.layers.ReLU(negative_slope=0.02)
    for i in range(hp.Int('layers', 2, 7)):
        model.add(Dense(units=hp.Int(f'units_{i}', 4, 200, step=8), activation=prelu))
    model.add(Dense(variables_to_predict))
    optimizer = tf.keras.optimizers.Adam(learning_rate=hp.Choice("learning_rate", [1e-3, 1e-4, 1e-5]))
    model.compile(loss='mse', optimizer=optimizer, metrics=["mae", "mse"])
    model.summary()

    return model

In [ ]:
if Execute_Train_flag:
    print("[INFO] instantiating a bayesian optimization tuner object...")
    tuner = kt.BayesianOptimization(  # kt.RandomSearch - kt.BayesianOptimization - kt.Hyperband
        build_model,
        objective="val_loss",
        max_trials=100,
        overwrite=False,
        directory=os.path.normpath('Model_optimized\\FCNN_v5\\BayesianOptimization_results'),
        project_name="Transonic_wing",
        seed=None)
  
    early_stop = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

    #Training The Artificial Neural Network

    # perform the hyperparameter search
    print("[INFO] performing hyperparameter search...")
    tuner.search(
    x=X_train, y=Y_train,
    validation_data=(X_test, Y_test),
    batch_size=6,
    callbacks=[early_stop],
    epochs=500,
    verbose=0
    )

    # grab the best hyperparameters
    bestHP = tuner.get_best_hyperparameters(num_trials=1)[0]

    early_stop_fit = EarlyStopping(monitor='val_loss', patience=300, restore_best_weights=True)

    print("[INFO] training the best model...")
    model = tuner.hypermodel.build(bestHP)
    history = model.fit(x=X_train, y=Y_train,
        validation_data=(X_test, Y_test),
        batch_size=6,
        callbacks=[early_stop_fit],
        epochs=10000,
        verbose=2)
    
    tuner.results_summary()

    print('Storing the trained model...')
    # Save the model:
    model.save(model_dir+'neural_network_model.h5')
    # Save the weights:
    model.save_weights(model_dir+'neural_network_weights.h5')


    # Plotting the training history
    loss = history.history['loss']
    mae = history.history['mae']
    mse = history.history['mse']
    val_loss = history.history['val_loss']
    val_mae = history.history['val_mae']
    val_mse = history.history['val_mse']


    plt.figure(figsize=(15,6))

    plt.subplot(1,3,1)
    plt.plot(loss,'b',label='training loss')
    plt.plot(val_loss,'r',label='validation loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend()
    plt.grid()
    # plt.ylim((0, 0.001))
    plt.title('training and validation loss evolution')

    plt.subplot(1,3,2)
    plt.plot(mae,'b',label='training mae')
    plt.plot(val_mae,'r',label='validation mae')
    plt.xlabel('epoch')
    plt.ylabel('mae')
    plt.legend()
    plt.grid()
    # plt.ylim((0, 0.01))
    plt.title('training and validation mae evolution')

    plt.subplot(1,3,3)
    plt.plot(mse,'b',label='training mse')
    plt.plot(val_mse,'r',label='validation mse')
    plt.xlabel('epoch')
    plt.ylabel('mse')
    plt.legend()
    plt.grid()
    # plt.ylim((0, 0.001))
    plt.title('training and validation mse evolution')

    plt.show()


In [ ]:
# Load or train model
if Transfer_Learning_flag:
    model = tf.keras.models.load_model(model_dir + 'neural_network_model.h5')
    model.load_weights(model_dir + 'neural_network_weights.h5')
    print(model.summary())

In [ ]:
# Prediction and inverse transform
Y_val_pred = model.predict(X_val, batch_size=1)

X_val = scaler.inverse_transform(X_val.reshape(-1, X.shape[2])).reshape(X_val.shape)
Y_val = scalery.inverse_transform(Y_val.reshape(-1, Y.shape[2])).reshape(Y_val.shape)
Y_val_pred = scalery.inverse_transform(Y_val_pred.reshape(-1, Y_val_pred.shape[1])).reshape(Y_val_pred.shape)